# Multivariate Linear Regression with Gradient Descent
## Implementing Multivariate Linear Regression
Multivariate linear regression is a more general form of unvariate linear regression. Instead of a single variable $x$ we use a feature vector $x = [x_1, x_2, ..., x_k]$ where $x_i$ are real-numbered features of our data. Our weight vector is $w = [w_0, w_1 w_2, ..., w_k]$

In multivariate linear regression our hypothesis space is $h_w(x) = w_0 + w_1x_1 + ... w_kx_k$ for all $w$.

We can make it simpler to calculate by adding a constant feature $x_0 = 1$. That way our hypothesis will be:

$$h_w(x) = w_0x_0 + w_1x_1 + ... w_kx_k = w \cdot x$$

We'll use the mean-squared-error loss function like we did in univariate linear regression. For a training set of size $n$ our loss function is:

$$L(h_w) = {1 \over n} \sum_{i=1}^n (y_i - h_w(x_i))^2 = {1 \over n} \sum_{i=1}^n (y_i - w \cdot x_i)^2$$

In code:

In [1]:
import numpy as np

# X is an n x k matrix. Y is an n-vector. W is a k-vector.
def meanSquaredError(X,Y,W):
    n, k = X.shape
    HX = np.matmul(X,W)
    ERR = (Y - HX) ** 2
    MSE = sum(ERR) / n
    return MSE

# The loss function is a closure over the training set.
def mkLossFunc(X,Y):
    def L(W): 
        return meanSquaredError(X,Y,W)
    return L

Unfortunately, it's not feasible to draw a contour plot of a $k$-dimensional loss function, but we can plot the data in tabular form.

Here's a sample of $L$ when $f(x) = 3 + 2x_1 + 5x_2 + x_3$:

In [2]:
import random

# Make up some feature vectors. 
X0 = np.repeat(1,50)

X1 = list(np.linspace(1,5))
random.shuffle(X1)
X1 = np.array(X1)

X2 = list(np.linspace(10,50))
random.shuffle(X2)
X2 = np.array(X2)

X3 = list(np.linspace(13, 45))
random.shuffle(X3)
X3 = np.array(X3)

# X is a 50 x 4 matrix.
X = np.array([X0,X1,X2,X3]).T

# Compute Y. Y is a 50-vector.
F = np.array([3,2,5,1])
Y = np.matmul(X,F)

# Make loss function.
L = mkLossFunc(X,Y)

# Make up some weight vectors. Only make weights with values 1, 2, 3, 4, 5 to keep it simple.
W = []
for i in range(1,6):
    for j in range(1,6):
        for k in range(1,6):
            for l in range(1,6):
                W.append([i,j,k,l])
W = np.array(W)

rows, cols = W.shape
for i in range(rows):
    w = W[i]
    loss = L(w)
    print(w, loss)

[1 1 1 1] 17816.7124531
[1 1 1 2] 11524.3774427
[1 1 1 3] 7091.67508538
[1 1 1 4] 4518.60538109
[1 1 1 5] 3805.16832986
[1 1 2 1] 10252.804748
[1 1 2 2] 5692.97419409
[1 1 2 3] 2992.77629321
[1 1 2 4] 2152.2110454
[1 1 2 5] 3171.27845065
[1 1 3 1] 4766.44806331
[1 1 3 2] 1939.12196585
[1 1 3 3] 971.428521449
[1 1 3 4] 1863.36773011
[1 1 3 5] 4614.93959184
[1 1 4 1] 1357.642399
[1 1 4 2] 262.820758017
[1 1 4 3] 1027.6317701
[1 1 4 4] 3652.07543524
[1 1 4 5] 8136.15175344
[1 1 5 1] 26.387755102
[1 1 5 2] 664.070570596
[1 1 5 3] 3161.38603915
[1 1 5 4] 7518.33416077
[1 1 5 5] 13734.9149354
[1 2 1 1] 17104.4081633
[1 2 1 2] 10988.2066639
[1 2 1 3] 6731.63781758
[1 2 1 4] 4334.70162432
[1 2 1 5] 3797.39808413
[1 2 2 1] 9712.97959184
[1 2 2 2] 5329.28254894
[1 2 2 3] 2805.2181591
[1 2 2 4] 2140.78642232
[1 2 2 5] 3335.98733861
[1 2 3 1] 4399.10204082
[1 2 3 2] 1747.90945439
[1 2 3 3] 956.349521033
[1 2 3 4] 2024.42224073
[1 2 3 5] 4952.12761349
[1 2 4 1] 1162.7755102
[1 2 4 2] 244.087380258


Just like in univariate linear regression, the update function we'll use for linear regression is:

$$w_k = w_k - \alpha {\delta \over \delta w_k} L(h_w) = w_i + {2 \alpha \over n} \sum_{i=1}^n (y_i - h_w(x_i))x_{i,j} $$

In code:

In [3]:
# X is an n x k matrix, Y is an n-vector, and alpha is the learning rate.
def mkUpdateFunc(X,Y,alpha):
    n, k = X.shape
    
    # W is a k-vector.
    def updateW(W):
        learningRate = 2 * alpha / n # scalar
        ERR = Y - np.matmul(X,W) # n-vector
        GRADIENTS = np.matmul(X.T,ERR)
        return W + learningRate * GRADIENTS
    return updateW

Now we implement gradient descent with our new update function:

In [4]:
# X is an n x k matrix. Y is an n-vector. alpha is the learning rate.
def gradientDescent(X,Y,alpha,omega):
    n, k = X.shape
    W = np.repeat(0,k)
    L = mkLossFunc(X,Y)
    updateW = mkUpdateFunc(X,Y,alpha)
    loss = L(W)
    while(True):
        W = updateW(W)
        newLoss = L(W)
        if abs(newLoss - loss) < omega:
            break
        else:
            loss = newLoss
    return W

def fitLine(X,Y,alpha,omega):
    W = gradientDescent(X,Y,alpha,omega)
    L = mkLossFunc(X,Y)
    print("weights: ", W)
    print("loss: ", L(W))  
    return W
    
# Alternative implementation of gradient descent that is guaranteed to terminate after a set number of iterations.
# This is useful for feeling out a good value for alpha.

# epochs is the number of iterations.
def gradientDescentTerminating(X,Y,alpha,epochs):
    n, k = X.shape
    W = np.repeat(0,k)
    updateW = mkUpdateFunc(X,Y,alpha)
    while(epochs > 0):
        W = updateW(W)
        epochs -= 1
    return W

def fitLineTerminating(X,Y,alpha):
    W = gradientDescentTerminating(X,Y,alpha,10000)
    L = mkLossFunc(X,Y)
    print("weights: ", W)
    print("loss: ", L(W)) 
    return W

## Evaluating Multivariate Linear Regression on Known Functions
Here's what happens when we use multivariate linear regression on a dataset generated by $f(x) = 3 + 2x_i + 5x_2 + x_3$:

In [5]:
import random

# Make up some feature vectors. 
X0 = np.repeat(1,50)

X1 = list(np.linspace(1,5))
random.shuffle(X1)
X1 = np.array(X1)

X2 = list(np.linspace(10,50))
random.shuffle(X2)
X2 = np.array(X2)

X3 = list(np.linspace(13, 45))
random.shuffle(X3)
X3 = np.array(X3)

# X is a 50 x 4 matrix.
X = np.array([X0,X1,X2,X3]).T

# Compute Y. Y is a 50-vector.
F = np.array([3,2,5,1])
Y = np.matmul(X,F)

alpha = 0.0001
omega = 0.00001
fitLine(X,Y,alpha,omega)

weights:  [ 0.50818634  2.1874116   5.02245444  1.03889339]
loss:  0.302169920465


array([ 0.50818634,  2.1874116 ,  5.02245444,  1.03889339])

Just like univariate linear regression, multivariate regression tends to underestimate $w_0$ and overestimate all the other weights to compensate. Still, loss is very small within the sample space.

## Evaluating Multivariate Linear Regression on Unknown Functions



To evaluate linear regression on an unkown function, we'll use U.S. Census data showing the county-level results in the 1992 election. 

The feature vector is:
* $x_1$ - Median Age (years)
* $x_2$ - Median Savings in Dollars
* $x_3$ - Per-Capita Income Dollars
* $x_4$ - Percent in Poverty
* $x_5$ - Percent Veterans
* $x_6$ - Percent Female
* $x_7$ - Population Density
* $x_8$ - Percent in Nursing Homes
* $x_9$ - Crime Index Per Capita

The output variable is the percentage of voters that voted for Bill Clinton.

In [7]:
import numpy as np
import pandas as pd
import random


DATA = pd.read_csv("election.csv").values
DATA = list(DATA)
random.shuffle(DATA)
DATA = np.array(DATA)
SPLITTED = np.array_split(DATA,2)
TRAIN = SPLITTED[0].T
TEST = SPLITTED[1].T

TRAIN_Y = TRAIN[0]
TRAIN_X = TRAIN[1:].T
TEST_Y = TEST[0]
TEST_X = TEST[1:].T

alpha = 0.00000000009
omega = 0.001
print("TRAINING RESULTS")
W = fitLine(TRAIN_X,TRAIN_Y,alpha,omega)
print()
print("TEST RESULTS")
loss = mkLossFunc(TEST_X,TEST_Y)(W)
print("loss: ", loss)



TRAINING RESULTS
weights:  [  1.03718846e-05   4.15806394e-06   2.27030520e-03   1.10443383e-05
   2.93986949e-06   1.67390893e-05   6.14576323e-05   1.80080493e-06
   9.63616310e-05]
loss:  197.981299745

TEST RESULTS
loss:  208.331199255


The loss is large, suggesting that the true function is not linear, but the difference is loss between the training and test sets is small. Our hypothesis isn't as consistent with the data as we'd like it to be, but it does generalize which suggests that it does tell us something useful.